In [8]:
# === Master Settings ===
MODEL_PATH = "Agesture_Best.keras"

# Data and training
image_size = (128, 128)

# Freezing settings
freeze_layer_names = ['conv2d','conv2d_1']  # Freeze Conv1, Conv2

# Regularization
l2_strength = 0.00001  # L2 regularization strength for Conv layers

# MaxPooling adjustment
use_maxpooling = True  # Set to False if you want to disable MaxPooling layers
dropout = 0.4  # Adjust dropout rate
#dropout_rates = {
#    'dropout': 0.2,
#    'dropout_1': 0.3,
#    'dropout_2': 0.5
#}



In [9]:
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dropout, MaxPooling2D
import os
import json
import numpy as np

# === Load base model ===
base_model = load_model(MODEL_PATH)

def rebuild_model_with_modifications(base_model, freeze_names, l2_strength, dropout_rate, use_maxpool=True):
    from tensorflow.keras import regularizers
    from tensorflow.keras.layers import Dropout, MaxPooling2D, Layer, InputLayer

    class Identity(Layer):
        def call(self, inputs):
            return inputs

    new_layers = []

    for layer in base_model.layers:
        config = layer.get_config()
        name = layer.name
        layer_type = type(layer)

        # Only apply L2 to Conv3 & Conv4
        apply_l2 = name in ['conv2d_2']
        apply_drop = name in []
        

        try:
            if isinstance(layer, tf.keras.layers.Conv2D):
                new_layer = tf.keras.layers.Conv2D(
                    filters=config['filters'],
                    kernel_size=config['kernel_size'],
                    strides=config['strides'],
                    padding=config['padding'],
                    activation=config['activation'],
                    kernel_regularizer=regularizers.l2(l2_strength) if apply_l2 else None,
                    name=name
                )

            elif isinstance(layer, tf.keras.layers.Dropout):
                # Rebuild Dropout with new rate
                new_layer = Dropout(dropout_rate, name=layer.name) if apply_drop else None,
            #elif isinstance(layer, Dropout):
            #    rate = dropout_rate.get(name, 0.5) if isinstance(dropout_rate, dict) else dropout_rate
            #    new_layer = Dropout(rate, name=name)

            elif isinstance(layer, MaxPooling2D):
                if use_maxpool:
                    new_layer = MaxPooling2D(
                        pool_size=config['pool_size'],
                        strides=config['strides'],
                        padding=config['padding'],
                        name=name
                    )
                else:
                    new_layer = Identity(name=name + "_skip")
            elif isinstance(layer, InputLayer):
                new_layer = tf.keras.layers.InputLayer(input_shape=config['batch_input_shape'][1:], name=name)
            else:
                new_layer = layer_type.from_config(config)

            new_layer.trainable = name not in freeze_names
            new_layers.append(new_layer)

        except Exception as e:
            print(f"⚠️ Failed to rebuild layer '{name}' ({layer_type}): {e}")
            continue

    # Reconnect layers
    inputs = tf.keras.Input(shape=image_size + (3,))
    x = inputs
    for layer in new_layers:
        try:
            x = layer(x)
        except Exception as e:
            print(f"❌ Failed to apply layer '{layer.name}' to input tensor: {e}")
            raise e

    return tf.keras.Model(inputs=inputs, outputs=x)

# === Build new model ===
new_model = rebuild_model_with_modifications(
    base_model,
    freeze_layer_names,
    l2_strength,
    dropout,
    use_maxpooling
)

print("\n🛠️ New Model Summary:")
new_model.summary()

# === Debug model freeze and regularization ===
def debug_model_freezing(model):
    print(f"{'Layer Name':25} | {'Trainable':10} | {'Regularizer'}")
    print("-" * 60)
    for layer in model.layers:
        reg = getattr(layer, "kernel_regularizer", None)
        print(f"{layer.name:25} | {str(layer.trainable):10} | {str(reg)}")

debug_model_freezing(new_model)


⚠️ Failed to rebuild layer 'input_layer_1' (<class 'keras.src.layers.core.input_layer.InputLayer'>): 'batch_input_shape'
⚠️ Failed to rebuild layer 'dropout' (<class 'keras.src.layers.regularization.dropout.Dropout'>): 'tuple' object has no attribute 'trainable'
⚠️ Failed to rebuild layer 'dropout_1' (<class 'keras.src.layers.regularization.dropout.Dropout'>): 'tuple' object has no attribute 'trainable'
⚠️ Failed to rebuild layer 'dropout_2' (<class 'keras.src.layers.regularization.dropout.Dropout'>): 'tuple' object has no attribute 'trainable'

🛠️ New Model Summary:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 128, 128, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 64, 64, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 65536)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       8,388,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,456,487 (32.26 MB)

 Trainable params: 8,445,703 (32.22 MB)

 Non-trainable params: 10,784 (42.12 KB)

Layer Name                | Trainable  | Regularizer
------------------------------------------------------------
input_layer_1             | True       | None
conv2d                    | False      | None
batch_normalization       | True       | None
conv2d_1                  | False      | None
batch_normalization_1     | True       | None
max_pooling2d             | True       | None
conv2d_2                  | True       | <keras.src.regularizers.regularizers.L2 object at 0x0000013BD1545540>
batch_normalization_2     | True       | None
conv2d_3                  | True       | None
batch_normalization_3     | True       | None
max_pooling2d_1           | True       | None
flatten                   | True       | None
dense                     | True       | None
batch_normalization_4     | True       | None
dense_1                   | True       | None


In [10]:
new_model.save("Bgesture_CNN.keras")
print("✅ Modified Model saved as gesture_CNN_Modified.keras")

✅ Modified Model saved as gesture_CNN_Modified.keras


In [11]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)


0 input_layer_1
1 conv2d
2 batch_normalization
3 conv2d_1
4 batch_normalization_1
5 max_pooling2d
6 dropout
7 conv2d_2
8 batch_normalization_2
9 conv2d_3
10 batch_normalization_3
11 max_pooling2d_1
12 dropout_1
13 flatten
14 dense
15 batch_normalization_4
16 dropout_2
17 dense_1


In [12]:
def print_frozen_layers(model):
    print(f"{'Layer Name':25} | {'Trainable':10}")
    print("-" * 40)
    for layer in model.layers:
        print(f"{layer.name:25} | {str(layer.trainable):10}")


In [13]:
print_frozen_layers(base_model)


Layer Name                | Trainable 
----------------------------------------
input_layer_1             | True      
conv2d                    | True      
batch_normalization       | True      
conv2d_1                  | True      
batch_normalization_1     | True      
max_pooling2d             | True      
dropout                   | True      
conv2d_2                  | False     
batch_normalization_2     | True      
conv2d_3                  | False     
batch_normalization_3     | True      
max_pooling2d_1           | True      
dropout_1                 | True      
flatten                   | True      
dense                     | True      
batch_normalization_4     | True      
dropout_2                 | True      
dense_1                   | True      


In [14]:
for i, layer in enumerate(new_model.layers):
    print(i, layer.name)


0 input_layer_1
1 conv2d
2 batch_normalization
3 conv2d_1
4 batch_normalization_1
5 max_pooling2d
6 conv2d_2
7 batch_normalization_2
8 conv2d_3
9 batch_normalization_3
10 max_pooling2d_1
11 flatten
12 dense
13 batch_normalization_4
14 dense_1
